In [1]:
import os
import json
import easydict
from easydict import EasyDict
import numpy as np
from tqdm import tqdm

In [ ]:
raw_path = "data/charades_reset/train.json"
with open(raw_path, 'r') as fr:
    raw_json = json.load(fr)
new_json = []
for i in raw_json:
    new_json.append(i[:4])
    assert i[1] >= i[2][1], "{} {}".format( i[1], i[2][1])


new_path = "data/charades_clean/train.json"
with open(new_path, 'w') as fr:
    json.dump(new_json, fr)

In [ ]:
config_path = "config/charades/main_i3d.json"

In [ ]:
config_path = "config/charades/main_i3d.json"
def load_json(filename):
    with open(filename, encoding='utf8') as fr:
        return EasyDict(json.load(fr))


In [ ]:
with open(config_path, encoding='utf8') as fr:
    a  = json.load(fr)

In [ ]:
feature_dir = "/storage/rjliang/W/VSLNet/data/features/charades/new" # 6673
feat_len_path = os.path.join(feature_dir, 'feature_shapes.json')
vfeat_lens = load_json(feat_len_path)
# vfeat_lens = get_vfeat_len(feature_dir)

In [ ]:
feature_dir2 = "/storage_fast/rjliang/charades/c3d_1024"
lens2 = {}
v_list = os.listdir(feature_dir2)

In [ ]:
for vid in v_list:
    tmp = os.path.join(feature_dir2, vid)
    ll = np.load(tmp).shape[0]
    lens2[vid[:-4]] = ll
    break

In [ ]:
print(len(vfeat_lens))
print(len(lens2))



In [ ]:
for vid in vfeat_lens:
    assert vfeat_lens[vid] == lens2[vid]

In [ ]:

def compute_overlap(pred, gt):
    # check format
    assert isinstance(pred, list) and isinstance(gt, list)
    pred_is_list = isinstance(pred[0], list)
    gt_is_list = isinstance(gt[0], list)
    pred = pred if pred_is_list else [pred]
    gt = gt if gt_is_list else [gt]
    # compute overlap
    pred, gt = np.array(pred), np.array(gt)
    inter_left = np.maximum(pred[:, 0, None], gt[None, :, 0])
    inter_right = np.minimum(pred[:, 1, None], gt[None, :, 1])
    inter = np.maximum(0.0, inter_right - inter_left)
    union_left = np.minimum(pred[:, 0, None], gt[None, :, 0])
    union_right = np.maximum(pred[:, 1, None], gt[None, :, 1])
    union = np.maximum(1e-12, union_right - union_left)
    overlap = 1.0 * inter / union
    # reformat output
    overlap = overlap if gt_is_list else overlap[:, 0]
    overlap = overlap if pred_is_list else overlap[0]
    return overlap


def time_to_index(start_time, end_time, num_units, duration):
    s_times = np.arange(0, num_units).astype(np.float32) / float(num_units) * duration
    e_times = np.arange(1, num_units + 1).astype(np.float32) / float(num_units) * duration
    candidates = np.stack([np.repeat(s_times[:, None], repeats=num_units, axis=1),
                           np.repeat(e_times[None, :], repeats=num_units, axis=0)], axis=2).reshape((-1, 2))
    overlaps = compute_overlap(candidates.tolist(), [start_time, end_time]).reshape(num_units, num_units)
    start_index = np.argmax(overlaps) // num_units
    end_index = np.argmax(overlaps) % num_units
    return start_index, end_index, overlaps

In [ ]:
def get_vfeat_len(feature_dir, max_pos_len):
    vfeat_lens = {}
    for vid in tqdm(os.listdir(feature_dir), desc="get video feature lengths"):
        tmp = os.path.join(feature_dir, vid)
        ll = np.load(tmp).shape[0]
        vfeat_lens[vid[:-4]] = min(max_pos_len, ll)
    return vfeat_lens 

In [ ]:
raw_path =  "data/charades_clean/train.json"
with open(raw_path, 'r') as fr:
    raw_json = json.load(fr)
vfeat_lens = get_vfeat_len(feature_dir="/storage_fast/rjliang/charades/c3d_1024", max_pos_len=64)
 

In [ ]:
def time_to_index_my(s, e,  feature_len, duration):
    s = round(s / duration * feature_len)
    e = round(e / duration * feature_len)
    return s, e

for idx, record in enumerate(raw_json):
    vid = record[0]
    duration = record[1]
    s, e = record[2]
    flen = vfeat_lens[vid]
    s_ind1, e_ind1, _ = time_to_index(s, e, flen, duration)
    s_ind2, e_ind2 = time_to_index_my(s, e, flen, duration)
    
    assert e_ind1 <= flen
    assert e_ind2 <= flen
    assert s_ind1 == s_ind2, "{} {}".format(s_ind1, s_ind2)
    assert e_ind1 == e_ind2, "{} {} {}".format(e_ind1, e_ind2, idx)
    


In [6]:
def time_to_index_my(s, e,  feature_len, duration):
    feature_len = feature_len - 1
    s = round(s / duration * feature_len)
    e = round(e / duration * feature_len)
    return s, e


for i in range(100):

    s = 0.0
    e = 0.0 + i*0.1
    feature_nums = 11
    duration = 10

    s_ind1, e_ind1, _ = time_to_index(s, e, feature_nums, duration)
    s_ind2, e_ind2 = time_to_index_my(s, e, feature_nums, duration)

    print("{:.2f} {:.2f}".format(s, e), end="")
    print("|", s_ind1, e_ind1, "|", s_ind2, e_ind2)


# assert s_ind1 == s_ind2, "{} {}".format(s_ind1, s_ind2)
# assert e_ind1 == e_ind2, "{} {} {}".format(e_ind1, e_ind2, idx)



0.00 0.00| 0 0 | 0 0
0.00 0.10| 0 0 | 0 0
0.00 0.20| 0 0 | 0 0
0.00 0.30| 0 0 | 0 0
0.00 0.40| 0 0 | 0 0
0.00 0.50| 0 0 | 0 1
0.00 0.60| 0 0 | 0 1
0.00 0.70| 0 0 | 0 1
0.00 0.80| 0 0 | 0 1
0.00 0.90| 0 0 | 0 1
0.00 1.00| 0 0 | 0 1
0.00 1.10| 0 0 | 0 1
0.00 1.20| 0 0 | 0 1
0.00 1.30| 0 1 | 0 1
0.00 1.40| 0 1 | 0 2
0.00 1.50| 0 1 | 0 2
0.00 1.60| 0 1 | 0 2
0.00 1.70| 0 1 | 0 2
0.00 1.80| 0 1 | 0 2
0.00 1.90| 0 1 | 0 2
0.00 2.00| 0 1 | 0 2
0.00 2.10| 0 1 | 0 2
0.00 2.20| 0 1 | 0 2
0.00 2.30| 0 2 | 0 3
0.00 2.40| 0 2 | 0 3
0.00 2.50| 0 2 | 0 3
0.00 2.60| 0 2 | 0 3
0.00 2.70| 0 2 | 0 3
0.00 2.80| 0 2 | 0 3
0.00 2.90| 0 2 | 0 3
0.00 3.00| 0 2 | 0 3
0.00 3.10| 0 2 | 0 3
0.00 3.20| 0 3 | 0 4
0.00 3.30| 0 3 | 0 4
0.00 3.40| 0 3 | 0 4
0.00 3.50| 0 3 | 0 4
0.00 3.60| 0 3 | 0 4
0.00 3.70| 0 3 | 0 4
0.00 3.80| 0 3 | 0 4
0.00 3.90| 0 3 | 0 4
0.00 4.00| 0 3 | 0 4
0.00 4.10| 0 4 | 0 5
0.00 4.20| 0 4 | 0 5
0.00 4.30| 0 4 | 0 5
0.00 4.40| 0 4 | 0 5
0.00 4.50| 0 4 | 0 5
0.00 4.60| 0 4 | 0 5
0.00 4.70| 0 

In [2]:
import torch
logits = torch.randn(20, 32)
gumbels = (-torch.empty_like(logits, memory_format=torch.legacy_contiguous_format).exponential_().log())

/storage/rjliang/anaconda3/envs/py3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.orthogonal(m.weight)

NameError: name 'legacy_contiguous_format' is not defined

In [42]:
import torch
from torch import nn


# label_embs = tf.get_variable(name='label_emb', shape=[4, self.configs.dim], dtype=tf.float32,
#                                     trainable=True, initializer=tf.orthogonal_initializer())
# ortho_constraint = tf.multiply(tf.matmul(label_embs, label_embs, transpose_b=True),
#                                 1.0 - tf.eye(4, dtype=tf.float32))

label_embs = torch.empty(size=[128, 4], dtype=torch.float32)
label_embs = nn.init.orthogonal(label_embs.data)
ortho_constraint = torch.matmul(label_embs.T, label_embs) * (1.0 - torch.eye(4, dtype=torch.float32))




/tmp/ipykernel_353010/2853727174.py:11: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  label_embs = nn.init.orthogonal(label_embs.data)


In [43]:
print(label_embs[0])
print(label_embs[1])

tensor([ 0.0390,  0.1874, -0.1183,  0.0888])
tensor([ 0.1158, -0.1336,  0.0822,  0.0716])


In [44]:
ortho_constraint = torch.norm(ortho_constraint, p=2)  # compute l2 norm as loss
# self.match_loss += ortho_constraint


In [45]:
label_embs.shape

torch.Size([128, 4])

In [46]:
B = 8
L = 64
D = 128
match_scores = torch.rand(B, L, 4)
torch.matmul(match_scores, torch.tile(label_embs, (8, 1, 1)).permute(0, 2, 1)).shape

torch.Size([8, 64, 128])

In [41]:
label_embs.shape

torch.Size([64, 4])

In [34]:
torch.repeat_interleave(label_embs)

RuntimeError: "repeat_interleave_cpu" not implemented for 'Float'

In [33]:
label_embs.shape

torch.Size([64, 4])